In [1]:
import os
os.environ["R_HOME"] = "C:/PROGRA~1/R/R-44~1.2"

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
splatter = importr("splatter")

%load_ext rpy2.ipython

c:\Users\Broondan\AppData\Local\Programs\Python\Python312\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [ ]:
%%R
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("splatter")

Update all/some/none? [a/s/n]: package 'assorthead' successfully unpacked and MD5 sums checked
package 'BiocNeighbors' successfully unpacked and MD5 sums checked
package 'cpp11' successfully unpacked and MD5 sums checked
package 'curl' successfully unpacked and MD5 sums checked
package 'edgeR' successfully unpacked and MD5 sums checked
package 'textshaping' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Broondan\AppData\Local\Temp\RtmpsfkpcX\downloaded_packages


Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31 ucrt)
Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31 ucrt)
Installation paths not writeable, unable to update packages
  path: C:/Program Files/R/R-4.4.2/library
  packages:
    cluster
Old packages: 'assorthead', 'BiocNeighbors', 'cpp11', 'curl', 'edgeR',
  'GenomeInfoDb', 'IRanges', 'textshaping'
trying URL 'https://bioconductor.org/packages/3.20/bioc/bin/windows/contrib/4.4/assorthead_1.0.1.zip'
Content type 'application/zip' length 2257782 bytes (2.2 MB)
downloaded 2.2 MB

trying URL 'https://bioconductor.org/packages/3.20/bioc/bin/windows/contrib/4.4/BiocNeighbors_2.0.1.zip'
Content type 'application/zip' length 1048980 bytes (1.0 MB)
downloaded 1.0 MB

trying URL 'https://cloud.r-project.org/bin/windows/contrib/4.4/cpp11_0.5.1.zip'
Content type 'application/zip' length 310609 bytes (303 KB)
downloaded 303 KB

trying URL 'https://cloud.r-project.org/bin/windows/contrib/4.4/curl_6.0.1.zip'
Co

In [12]:
%%R
install.packages("repr")

BiocManager::install("rhdf5")


[1] "\n\n"


package 'Rhdf5lib' successfully unpacked and MD5 sums checked
package 'rhdf5filters' successfully unpacked and MD5 sums checked
package 'rhdf5' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Broondan\AppData\Local\Temp\RtmpsfkpcX\downloaded_packages
Update all/some/none? [a/s/n]: 

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cloud.r-project.org
Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31 ucrt)
Installing package(s) 'rhdf5'
also installing the dependencies 'Rhdf5lib', 'rhdf5filters'

trying URL 'https://bioconductor.org/packages/3.20/bioc/bin/windows/contrib/4.4/Rhdf5lib_1.28.0.zip'
Content type 'application/zip' length 6544772 bytes (6.2 MB)
downloaded 6.2 MB

trying URL 'https://bioconductor.org/packages/3.20/bioc/bin/windows/contrib/4.4/rhdf5filters_1.18.0.zip'
Content type 'application/zip' length 2339841 bytes (2.2 MB)
downloaded 2.2 MB

trying URL 'https://bioconductor.org/packages/3.20/bioc/bin/windows/contrib/4.4/rhdf5_2.50.0.zip'
Content type 'application/zip' length 4579844 bytes (4.4 MB)
downloaded 4.4 MB

Installation paths not writeable, unable to update packages
  path: C:/Program Files/R/R-4.4.2/

In [14]:
%%R
library(splatter)
library(rhdf5)

set.seed(123)

params <- newSplatParams()
sim <- splatSimulate(params)

counts <- as.data.frame(t(counts(sim)))
true_counts <- as.data.frame(t(assays(sim)$TrueCounts))
dropout <- assays(sim)$Dropout
mode(dropout) <- 'integer'
cell_info <- as.data.frame(colData(sim))
gene_info <- as.data.frame(rowData(sim))

X <- t(counts)
Y <- as.integer(substring(cell_info$Group,6))
Y <- Y-1


# Tutorial code:
# sim <- splatSimulate(group.prob=group.prob, nGenes=nGenes, batchCells=batchCells,
#                     dropout.type="experiment", method=method,
#                     seed=100+i, dropout.shape=-1, dropout.mid=dropout, de.facScale=0.25)

Getting parameters...
Creating simulation object...
Simulating library sizes...
Simulating gene means...
Simulating BCV...
Simulating counts...
Simulating dropout (if needed)...
Sparsifying assays...
Automatically converting to sparse matrices, threshold = 0.95
Skipping 'BatchCellMeans': estimated sparse size 1.5 * dense matrix
Skipping 'BaseCellMeans': estimated sparse size 1.5 * dense matrix
Skipping 'BCV': estimated sparse size 1.5 * dense matrix
Skipping 'CellMeans': estimated sparse size 1.49 * dense matrix
Skipping 'TrueCounts': estimated sparse size 1.64 * dense matrix
Skipping 'counts': estimated sparse size 1.64 * dense matrix
Done!


In [ ]:
%%R


In [15]:
%%R
#### make tSNE plot for the simulated data
library(Rtsne)
library(ggplot2)
library(repr)
options(repr.plot.width=3, repr.plot.height=3)

X.normalized <- apply(X, 2, function(z) z/sum(z))
X.normalized <- log(X.normalized + 1)
tsne.X <- Rtsne(t(X.normalized), max_iter = 1000)
  
tsne_plot.X <- data.frame(`x-tsne` = tsne.X$Y[,1], `y-tsne` = tsne.X$Y[,2], 
                        truelabel = Y, check.names = F)
tsne_plot.X$truelabel <- factor(tsne_plot.X$truelabel, levels = c(0:max(Y)))

ggplot(tsne_plot.X) + geom_point(aes(x=`x-tsne`, y=`y-tsne`, color=truelabel), size=0.5) +
          labs(color='true label') +
          ggtitle("Simulated data") +
          theme_bw() +
          theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), 
                legend.key = element_rect(fill = 'white', colour = 'white'), legend.position="none",
                axis.title.y=element_blank(), axis.title.x=element_blank())

Error in data.frame(`x-tsne` = tsne.X$Y[, 1], `y-tsne` = tsne.X$Y[, 2],  : 
  arguments imply differing number of rows: 100, 0


RInterpreterError: Failed to parse and evaluate line '#### make tSNE plot for the simulated data\nlibrary(Rtsne)\nlibrary(ggplot2)\nlibrary(repr)\noptions(repr.plot.width=3, repr.plot.height=3)\n\nX.normalized <- apply(X, 2, function(z) z/sum(z))\nX.normalized <- log(X.normalized + 1)\ntsne.X <- Rtsne(t(X.normalized), max_iter = 1000)\n  \ntsne_plot.X <- data.frame(`x-tsne` = tsne.X$Y[,1], `y-tsne` = tsne.X$Y[,2], \n                        truelabel = Y, check.names = F)\ntsne_plot.X$truelabel <- factor(tsne_plot.X$truelabel, levels = c(0:max(Y)))\n\nggplot(tsne_plot.X) + geom_point(aes(x=`x-tsne`, y=`y-tsne`, color=truelabel), size=0.5) +\n          labs(color=\'true label\') +\n          ggtitle("Simulated data") +\n          theme_bw() +\n          theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), \n                legend.key = element_rect(fill = \'white\', colour = \'white\'), legend.position="none",\n                axis.title.y=element_blank(), axis.t